# Predección del semestre en cual un estudiante abandona sus estudios de pregrado

<img src="logo_udea.png" alt="drawing" width="100"/>



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas import read_excel
from sklearn.model_selection import train_test_split

## Leyendo los datos

In [2]:
my_sheet = 'SinDuplicadosDepurada' # change it to your sheet name
file_name = 'BDFinal.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name=my_sheet)
df.shape

(6593, 64)

En la siguiente fila se filtra por sede y modalidad.

In [3]:
df = df[(df.SEDE == 'MEDELLIN') & (df.MODALIDAD == 'PRESENCIAL')]


En la siguiente línea se crea la variable `y` categórica.

In [4]:
df['y'] = pd.Categorical(df['NIVEL_PREGRADO'])

In [5]:
pd.crosstab(index=df['y'], columns="Frecuencia")

col_0,Frecuencia
y,
1,3482
2,714
3,384
4,199
5,117
6,61
7,24
8,9
9,5


## Creación de los datos de entrenamiento (train) y de validación (test)
Para particionar los datos originales se usa la función `train_test_split`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df[['PROMEDIO_ULTIMO_SEMESTRE_TERMINADO',
                                                        'VERSION',
                                                        'CREDITOS_ULTIM_SEMEST_MATRIC',
                                                        'IND_POBREZA',
                                                        'Puntaje_total_estan',
                                                        'Puntaje_global',
                                                        'estu_edad']], 
                                                    df['y'], test_size=0.20)

## Ajuste del modelo SVM

Para crear el modelo se usa la función `sklearn.svm.SVC`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](https://scikit-learn.org/0.16/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [7]:
cw = {}
for i in set(y_train):
    cw[i] = np.sum(y_train == i)
print(cw)

{1: 2773, 2: 577, 3: 295, 4: 170, 5: 96, 6: 53, 7: 21, 8: 6, 9: 4, 10: 15, 11: 5}


In [8]:
from sklearn.svm import SVC

model = SVC(class_weight=cw, coef0=0.0, degree=3, gamma='auto', kernel='linear')
model_fit = model.fit(X=X_train, y=y_train)

Para obtener la tabla de confusión hacemos lo siguiente:

In [9]:
t1 = pd.crosstab(index=model.predict(X_train), columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])

print('\n')
print('La tabla de confusión usando train es: \n')
print(t1)

print('\n')
print('La tabla de confusión usando test es: \n')
print(t2)



La tabla de confusión usando train es: 

Membresia     1    2    3    4   5   6   7   8   9   10  11
Prediccion                                                 
1           2773  577  295  170  96  53  21   6   4  15   5


La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10  11
Prediccion                                              
1           709  137  89  29  21   8   3   3   1   3   1


In [10]:
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))

El Accuracy con test es: 0.6906600249066003
El Accuracy con train es: 0.7061752988047809


## Ajuste del modelo KNN

In [11]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=10)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10  11
Prediccion                                              
1           698  134  86  29  21   8   3   3   1   3   1
2            10    3   2   0   0   0   0   0   0   0   0
3             1    0   1   0   0   0   0   0   0   0   0


El Accuracy con test es: 0.6904109589041096
El Accuracy con train es: 0.6992031872509961


## Ajuste del modelo ARBOL

In [12]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1   2   3   4   5   6   7   8   9   10  11
Prediccion                                             
1           510  78  47  13  13   3   1   2   0   1   0
2            87  28  21   1   4   2   1   0   1   2   0
3            45  16   8   6   1   0   0   1   0   0   0
4            32   6   6   3   1   1   1   0   0   0   0
5            18   5   5   5   0   1   0   0   0   0   1
6            10   1   1   1   0   0   0   0   0   0   0
7             0   2   0   0   1   1   0   0   0   0   0
8             1   1   1   0   0   0   0   0   0   0   0
10            5   0   0   0   0   0   0   0   0   0   0
11            1   0   0   0   1   0   0   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.546812749003984


## Ajuste del modelo Gradient Tree Boosting

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10  11
Prediccion                                              
1           677  128  79  24  18   7   3   2   1   2   0
2             8    7   1   1   1   0   0   0   0   0   0
3             3    1   4   0   1   0   0   1   0   0   0
4             2    1   2   2   1   1   0   0   0   0   0
5             3    0   1   0   0   0   0   0   0   0   0
6             6    0   1   0   0   0   0   0   0   0   0
7             2    0   1   0   0   0   0   0   0   0   1
8             6    0   0   0   0   0   0   0   0   0   0
10            1    0   0   2   0   0   0   0   0   1   0
11            1    0   0   0   0   0   0   0   0   0   0


El Accuracy con test es: 0.7691158156911582
El Accuracy con train es: 0.6882470119521913


## Ajuste del modelo Forests of randomized trees¶

In [14]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1   2   3   4   5   6   7   8   9   10  11
Prediccion                                             
1           510  78  46  13  13   3   1   2   0   1   0
2            91  29  20   1   4   2   2   0   1   2   0
3            37  17   9   7   2   0   0   0   0   0   0
4            33   6   7   3   1   2   0   0   0   0   0
5            23   6   5   4   0   1   0   1   0   0   1
6            10   0   1   1   0   0   0   0   0   0   0
7             0   1   0   0   0   0   0   0   0   0   0
8             1   0   0   0   0   0   0   0   0   0   0
10            2   0   1   0   0   0   0   0   0   0   0
11            2   0   0   0   1   0   0   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.548804780876494


## Ajuste del modelo Multioutput classification

In [15]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 100, 100), max_iter=300)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10  11
Prediccion                                              
1           709  137  89  29  21   8   3   3   1   3   1


El Accuracy con test es: 0.6906600249066003
El Accuracy con train es: 0.7061752988047809


## Ajuste del modelo Stochastic Gradient Descent

In [16]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5000)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10  11
Prediccion                                              
1           709  137  89  29  21   8   3   3   1   3   1


El Accuracy con test es: 0.6906600249066003
El Accuracy con train es: 0.7061752988047809


## Un modelo adivinando

In [17]:
respuesta = pd.crosstab(index=df['y'], columns="Frecuencia")

import random

n = X_train.shape[0] # numero observaciones en train
y_al_azar = random.choices(population=respuesta.index.values, weights=respuesta['Frecuencia'], k=n)
y_al_azar = pd.Series(y_al_azar)

print('\n')
t2 = pd.crosstab(index=y_al_azar, columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
accu = t2.values.diagonal().sum() / t2.values.sum()
print('El Accuracy con train es:', accu)



La tabla de confusión usando test es: 

Membresia     1    2    3   4   5   6   7   8   9   10  11
Prediccion                                                
1           1071  255  137  92  58  38   6   4   2   6   3
2            218   51   27  17  14   5   5   0   2   3   0
3            110   24   13   8   5   5   5   1   0   2   0
4             52   17    7   7   4   1   1   1   0   0   1
5             37    8    9   2   1   0   1   0   0   0   0
6             22    1    1   3   1   0   0   0   0   0   0
7              8    2    0   3   0   0   0   0   0   0   1
8              5    0    0   0   0   0   0   0   0   0   0
9              2    1    0   0   0   0   0   0   0   0   0
10             4    1    1   1   0   0   0   0   0   0   0
11             7    0    0   0   0   0   0   0   0   0   0


El Accuracy con train es: 0.47625
